In [46]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can uncomment the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)

# !sudo ldconfig
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


openjdk-8-jdk-headless is already the newest version (8u432-ga~us1-0ubuntu2~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [47]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [48]:
rdd = sc.textFile("browsing.txt")

In [49]:
rdd.take(3)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ']

In [50]:
frequent_items_with_count = rdd.flatMap(lambda x: x.split(" ")).filter(lambda x: x != "").map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] >= 100).collect()

In [51]:
frequent_items_dict = {k: v for k, v in frequent_items_with_count}

In [52]:
def count_pairs(array):
  result = []
  for i in range(len(array)):
    for j in range(len(array)):
      if array[i] != array[j] and array[i] in frequent_items_dict.keys() and array[j] in frequent_items_dict.keys():
        result.append((array[i], array[j]))

  return result

In [53]:
frequent_pairs_with_count = rdd.map(lambda x: x.split(" ")).flatMap(count_pairs).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] >= 100).collect()

In [54]:
frequent_pairs_with_confidence_scores = [(pair, count / frequent_items_dict[pair[0]]) for pair, count in frequent_pairs_with_count]

In [55]:
frequent_pairs_with_confidence_scores.sort(key=lambda x: (-x[1], x[0]))

**The results of Part d:**

In [56]:
frequent_pairs_with_confidence_scores[: 5]

[(('DAI93865', 'FRO40251'), 1.0),
 (('GRO85051', 'FRO40251'), 0.999176276771005),
 (('GRO38636', 'FRO40251'), 0.9906542056074766),
 (('ELE12951', 'FRO40251'), 0.9905660377358491),
 (('DAI88079', 'FRO40251'), 0.9867256637168141)]

In [57]:
frequent_pairs_dict = {k: v for k, v in frequent_pairs_with_count}

In [58]:
def count_triples(array):
  result = []
  for i in range(len(array)):
    for j in range(len(array)):
      for k in range(len(array)):
        if array[i] != array[j] and array[i] != array[k] and array[j] != array[k] and (array[i], array[j]) in frequent_pairs_dict.keys() \
        and (array[i], array[k]) in frequent_pairs_dict.keys() and (array[j], array[k]) in frequent_pairs_dict.keys():
          result.append((array[i], array[j], array[k]))

  return result

In [59]:
frequent_triples_with_count = rdd.map(lambda x: x.split(" ")).flatMap(count_triples).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] >= 100).collect()

In [60]:
frequent_triples_with_confidence_scores = [(triple, count / frequent_pairs_dict[(triple[0], triple[1])]) for triple, count in frequent_triples_with_count]

In [61]:
frequent_triples_with_confidence_scores.sort(key=lambda x: (-x[1], x[0][0], x[0][1]))

**The results of Part e:**

In [62]:
frequent_triples_with_confidence_scores[: 5]

[(('DAI23334', 'ELE92920', 'DAI62779'), 1.0),
 (('DAI31081', 'GRO85051', 'FRO40251'), 1.0),
 (('DAI55911', 'GRO85051', 'FRO40251'), 1.0),
 (('DAI62779', 'DAI88079', 'FRO40251'), 1.0),
 (('DAI75645', 'GRO85051', 'FRO40251'), 1.0)]